In [1]:
# coding: UTF-8
import numpy as np
import cv2
import tkinter.filedialog as tkfd

# Line filter
buf_num = 10

# Read movie
clf_filename = tkfd.askopenfilename(title="select movie")
cap = cv2.VideoCapture(clf_filename)
cnt = 0

# Buffer initialization
ret, frame = cap.read()
line_image_buf = [0] * buf_num
for i in range(buf_num):
    line_image_buf[i] = np.copy(frame) * 0

# Convert setting
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('recognition_result_.avi',fourcc, 33.0, (frame.shape[1],frame.shape[0]))


while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Define a kernel size and apply Gaussian smoothing
        kernel_size = 5
        blur_gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)

        # Define our parameters for Canny and apply
        low_threshold = 50
        high_threshold = 150
        edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

        # Next we'll create a masked edges image using cv2.fillPoly()
        mask = np.zeros_like(edges)
        ignore_mask_color = 255

        # This time we are defining a four sided polygon to mask
        imshape = frame.shape

        left_bottom = [130, imshape[0]]
        right_bottom = [900, imshape[0]]
        left_top = [450, 320]
        right_top = [500, 320]

        vertices = np.array([[(left_bottom[0], left_bottom[1]), (left_top[0], left_top[1]), (right_top[0], right_top[1]), (right_bottom[0], right_bottom[1])]],
                                dtype=np.int32)
        cv2.fillPoly(mask, vertices, ignore_mask_color)
        masked_edges = cv2.bitwise_and(edges, mask)

        # Define the Hough transform parameters
        # Make a blank the same size as our image to draw on
        rho = 1  # distance resolution in pixels of the Hough grid
        theta = np.pi / 180  # angular resolution in radians of the Hough grid

        threshold = 30  # minimum number of votes (intersections in Hough grid cell)
        min_line_length = 100  # minimum number of pixels making up a line
        max_line_gap = 160  # maximum gap in pixels between connectable line segments

        line_image = np.copy(frame) * 0  # creating a blank to draw lines on
        lines_superimpose = np.copy(frame) * 0

        # Run Hough on edge detected image
        # Output "lines" is an array containing endpoints of detected line segments
        lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)


        # Iterate over the output "lines" and draw lines on a blank image
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 5)

        # Create a "color" binary image to combine with line image
        color_edges = np.dstack((edges, edges, edges))

        # Buffering line_images
        for i in reversed(range(buf_num)):
            line_image_buf[i] = line_image_buf[i-1]
        line_image_buf[0] = line_image

        # Draw the lines
        for i in reversed(range(buf_num)):
            lines_superimpose = cv2.addWeighted(lines_superimpose, 1, line_image_buf[i], 1, 0)
        lines_edges_fil = cv2.addWeighted(frame, 0.8, lines_superimpose, 1, 0)

        # Show movie
        out.write(lines_edges_fil)
        cv2.imshow('frame', lines_edges_fil)

        cnt += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()